In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.models.upload_data import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

### Limit Scoring

In [6]:
%%time
# Limit scoring engine
df_merged_scoring = limit_scoring(config_path)


Initial limits assignment:
limit_21_day   561798666.31
limit_7_day    559844097.29
limit_1_day    559844097.29
dtype: float64
Initial limits assignment sample:
      store_number  limit_21_day  limit_7_day  limit_1_day
33133       607795     127377.75     70057.76     70057.76
---------------------------------------------------------------------------------------------------------------------------------------

After limits decrease zeroization:
adjusted_21_limit   420031794.72
adjusted_7_limit    481337782.57
adjusted_1_limit    481337782.57
dtype: float64
After limits decrease zeroization sample:
      store_number  adjusted_21_limit  adjusted_7_limit  adjusted_1_limit
33133       607795               0.00              0.00              0.00
---------------------------------------------------------------------------------------------------------------------------------------

After limits zeroization till summary:
adjusted_21_limit   207007024.10
adjusted_7_limit    118972373.67
adj

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,any_bloom2_1day,total_outstanding_sum,safaricom_loan_balance_sum,loan_balance,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,rllvr_date_rm_ge_rm_add_back,never_borrowed_flag,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_21_limit,final_7_limit,final_1_limit
106255,254725570667,924499,2022-11-17,2022-12-11,12,219470.23,25,0.48,No_rules_relaxed,49.00,No,0.00,24558504,True,Reject,0.00,None,0,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,0.00,NaN,None,NaN,NaN,NaT,NaN,1.00,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,0,524900.00,562600.00,0,0,0,0.00,40500.00,40500.00,0,0,0,1,0,0.35,0.12,0.12,0.70,0.71,0.71,Band 2,Band 1,0.00,0.00,0.00,1.00,0.00,0,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
97965,254795166282,7886500,2022-11-19,2022-12-14,12,673.33,26,0.46,No_rules_relaxed,46.00,No,0.00,25804533,True,Reject,0.00,None,0,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,0.00,NaN,None,NaN,NaN,NaT,NaN,1.00,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,0,0.00,0.00,0,0,0,NaN,NaN,NaN,0,0,0,1,0,0.35,0.12,0.12,0.70,0.71,0.71,Band 2,Band 1,0.00,0.00,0.00,1.00,0.00,0,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 8.73 s
Wall time: 14.4 s


In [7]:
# Final dataset schema
df_merged_scoring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118921 entries, 0 to 118920
Data columns (total 94 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   mobile_number                            118401 non-null  object        
 1   store_number                             118921 non-null  object        
 2   most_recent_trx_date_past_30_days        102191 non-null  datetime64[ns]
 3   last_trx_date                            102191 non-null  datetime64[ns]
 4   actual_trx_days                          118921 non-null  int32         
 5   approx_30_days_trx_val                   118921 non-null  float64       
 6   expected_trx_days                        118921 non-null  int32         
 7   page_active_days                         118921 non-null  float64       
 8   inference_col                            118921 non-null  object        
 9   days_since_last_trx       

In [8]:
df = pd.read_parquet('../models/hurdle_rates_pred.parquet')
df.sample(2)

,cluster,repayment_milestone,hurdle_rate
13,4,erd,1.03
7,2,dpd90,0.73


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cluster              16 non-null     int64  
 1   repayment_milestone  16 non-null     object 
 2   hurdle_rate          16 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 512.0+ bytes


In [10]:
df.cluster.unique()

array([1, 2, 3, 4], dtype=int64)

In [11]:
df.repayment_milestone.unique()

array(['dd', 'erd', 'dpd30', 'dpd90'], dtype=object)